<a href="https://colab.research.google.com/github/surajit93/open-university-video/blob/main/open_university_video_renderer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!rm -rf *.wav *.png *.mp4 *.txt *.srt *.svg
print("Workspace cleaned")


In [ ]:
import os
os.environ["PHONEMIZER_ESPEAK_PATH"] = "/usr/lib/x86_64-linux-gnu/libespeak-ng.so.1"
os.environ["ESPEAK_PATH"] = "/usr/lib/x86_64-linux-gnu/libespeak-ng.so.1"
print("eSpeak configured")


In [ ]:
from TTS.api import TTS
import random

tts = TTS(model_name="tts_models/en/vctk/vits", progress_bar=False, gpu=False)
VOICE_POOL = ["p225","p226","p227","p228"]

def generate_dynamic_voice(text, out_file):
    speaker = random.choice(VOICE_POOL)
    text = text.replace(".", "... ")
    tts.tts_to_file(text=text, file_path=out_file, speaker=speaker)

print("Dynamic TTS engine ready")


In [ ]:
from pathlib import Path
import json

slide_plan = json.loads(Path("slide_plan.json").read_text())
audio_map  = json.loads(Path("slide_audio_map.json").read_text())

assert len(slide_plan["slides"]) == len(audio_map)
print("Slides loaded:", len(slide_plan["slides"]))


In [ ]:
import librosa
import numpy as np

def detect_energy_curve(audio_file):
    y, sr = librosa.load(audio_file, sr=None)
    energy = librosa.feature.rms(y=y)[0]
    return np.mean(energy)

def adaptive_chunk_duration(audio_file):
    energy = detect_energy_curve(audio_file)
    if energy > 0.08:
        return 3
    elif energy > 0.05:
        return 4
    else:
        return 5


In [ ]:
IMPORTANT_WORDS = [
    "money","power","rich","elite","secret","danger",
    "control","future","mistake","warning","truth",
    "hidden","win","lose","status","psychology"
]

def word_style(word):
    if word.lower() in IMPORTANT_WORDS:
        return {"color":"#FFD700","scale":1.3}
    return {"color":"white","scale":1.0}


In [ ]:
import subprocess

voice_files = []

for item in audio_map:
    sid  = item["slide_id"]
    text = item["spoken_text"]

    raw   = f"raw_{sid}.wav"
    final = f"voice_{sid}.wav"

    generate_dynamic_voice(text, raw)

    speed = random.uniform(0.95,1.1)

    subprocess.run([
        "ffmpeg","-y",
        "-i",raw,
        "-filter:a",f"atempo={speed}",
        "-ar","22050",
        "-ac","1",
        final
    ],check=True)

    voice_files.append(final)

print("Voice files ready")


In [ ]:
from moviepy.editor import AudioFileClip

def emotional_music(audio_file):
    energy = detect_energy_curve(audio_file)
    if energy > 0.08:
        volume = 0.25
    else:
        volume = 0.15
    return volume


In [ ]:
from moviepy.editor import VideoClip
import numpy as np

W,H = 1280,720

def zoom_background(duration):
    def make_frame(t):
        img = np.zeros((H,W,3),dtype=np.uint8)
        base = int(40 + (t*30)%100)
        img[:,:,0] = base
        img[:,:,1] = 40
        img[:,:,2] = 255-base
        return img
    clip = VideoClip(make_frame,duration=duration)
    return clip.resize(lambda t: 1+0.03*t)


In [ ]:
from moviepy.editor import TextClip

def animated_counter(start, end, duration):
    def make_text(t):
        value = int(start + (end-start)*(t/duration))
        return TextClip(str(value),
                        fontsize=120,
                        color="yellow",
                        font="DejaVu-Sans-Bold")
    return VideoClip(lambda t: make_text(t).get_frame(0),
                     duration=duration)


In [ ]:
def infographic_bar(value, duration):
    from moviepy.editor import VideoClip
    def make_frame(t):
        img = np.zeros((H,W,3),dtype=np.uint8)
        width = int((t/duration)*W*value)
        img[H//2-20:H//2+20,0:width] = (255,200,0)
        return img
    return VideoClip(make_frame,duration=duration)


In [ ]:
def moving_arrow(duration):
    def make_frame(t):
        img = np.zeros((H,W,3),dtype=np.uint8)
        x = int((t/duration)*W)
        img[H//2:H//2+10,x:x+100] = (255,0,0)
        return img
    return VideoClip(make_frame,duration=duration)


In [ ]:
from moviepy.editor import *
import math

def kinetic_text_layer(text,duration):
    words = text.split()
    clips=[]
    per = max(0.3,duration/len(words))
    t_cursor=0

    for w in words:
        style = word_style(w)
        clip=(TextClip(w,
                       fontsize=90,
                       font="DejaVu-Sans-Bold",
                       color=style["color"])
              .set_start(t_cursor)
              .set_duration(per)
              .resize(lambda t: style["scale"] + 0.2*math.sin(t*8))
              .set_position(("center","center")))

        clips.append(clip)
        t_cursor+=per

    return CompositeVideoClip(clips).set_duration(duration)


In [ ]:
segments=[]
import math

for slide in slide_plan["slides"]:
    sid  = slide["slide_id"]
    text = slide.get("right_panel_gist","")
    audio_file=f"voice_{sid}.wav"

    audio= AudioFileClip(audio_file)
    total=audio.duration
    chunk=adaptive_chunk_duration(audio_file)
    parts=math.ceil(total/chunk)

    for i in range(parts):
        start=i*chunk
        end=min((i+1)*chunk,total)
        dur=end-start

        bg = zoom_background(dur)
        txt = kinetic_text_layer(text,dur)
        arrow = moving_arrow(dur)

        scene = CompositeVideoClip([bg,arrow,txt])
        scene = scene.set_audio(audio.subclip(start,end))

        segments.append(scene.crossfadein(0.2))


In [ ]:
intro = TextClip("OPEN MEDIA UNIVERSITY",
                 fontsize=100,
                 color="white",
                 font="DejaVu-Sans-Bold")\
        .set_duration(3)\
        .fadein(1).fadeout(1)\
        .set_position("center")

outro = TextClip("SUBSCRIBE FOR NEXT EPISODE",
                 fontsize=80,
                 color="yellow",
                 font="DejaVu-Sans-Bold")\
        .set_duration(4)\
        .fadein(1)\
        .set_position("center")


In [ ]:
final_video = concatenate_videoclips(
    [intro] + segments + [outro],
    method="compose"
)

final_video.write_videofile("final.mp4",fps=30)
